In [1]:
import numpy as np
import os
import json
from pycocotools.coco import COCO
import json
import random
import re

In [3]:
## read annotations

file_path = "LVIS/lvis_v1_train.json"
with open(file_path, "r") as f:
    data = json.load(f)
coco=COCO(file_path)
print(len(data['annotations']))


loading annotations into memory...
Done (t=32.34s)
creating index...
index created!
1270141


In [6]:
##  creating image dictionaries and removing bad objects
image_dict = {}
for el in data['images']:
    cur_id = el['id']
    image_dict[cur_id] = el

obj_dict = {}
for el in data['categories']:
    name = el['name'].replace("_", " ")
    obj_dict[el['id']] = name


image_category = {}
for el in data['annotations']:
    img_id = el['image_id']
    cat_id = el['category_id']
   
    if img_id in image_category:
        if cat_id in image_category[img_id]:
            image_category[img_id][cat_id].append(el['id'])
        else:
            image_category[img_id][cat_id] = [el['id']]
    else:
        image_category[img_id] = {cat_id : [el['id']]}
        
for id, el in image_dict.items():
    if len(el['not_exhaustive_category_ids'])!= 0:
        for cat in el['not_exhaustive_category_ids']:
            del image_category[id][cat]
        if len(image_category[id]) == 0:
            del image_category[id]
print(len(image_dict), len(image_category) )    



100170 97346


In [16]:
## load annotations updated for 336x336 images
with open('annotations.json', 'r') as f:
    annotations = json.load(f)

## random 5000 image ids to use for training (only bbox generation) - these images have at least an object that has instances between 0-15
train_ids = np.load("train_ids.npy")
## random 250 image ids for CoT data 
cot_ids = np.load("cot_ids.npy")

## we used GPT-4o to get plurals for objects in LVIS
plurals = np.load("plurals.npy", allow_pickle=True).item()


In [17]:
### create only bbox data


image_dict = {}
for el in data['images']:
    cur_id = el['id']
    image_dict[cur_id] = el

n = set()
bads = 0

import random
random.seed(0)
items = list(image_dict.items())
random.shuffle(items)
image_dict = dict(items)
from collections import OrderedDict
image_dict = OrderedDict(image_dict)
question_dict = {}
count_dict = {}
alots = 0
H, W = 336, 336
for id , el in image_dict.items():
    if id not in train_ids:
        continue
    if id not in image_category:
        neg_cats = el['neg_category_ids']
        sample_cat = random.sample(neg_cats, 1)[0]
        object = obj_dict[sample_cat]
        plural =  plurals[object]
        a = "0"
        num =0 
        if num in count_dict and count_dict[num] >= 1000:
            continue
        if num not in count_dict:
            count_dict[num] =1
        else:
            count_dict[num] +=1
    
    else:
        cats = list(image_category[id].keys())
        nums = [len(image_category[id][cat]) for cat in list(image_category[id].keys())]
        index =  np.argmax(nums)
        num = nums[index]
        cat =  cats[index]
        ann_ids = image_category[id][cat]
        is_valid = all([str(t) in annotations for t in ann_ids])
        if not is_valid:
            bads+=1
            continue
        object = obj_dict[cat]
        plural =  plurals[object]
        assert num == len(image_category[id][cat])
        a = f"{str(num)}"
        if num > 15:
            alots +=1
            continue
        if num in count_dict and count_dict[num] >= 1000:
            continue
        if num not in count_dict:
            count_dict[num] =1
        else:
            count_dict[num] +=1
  
    if num !=0:
        answer = f"Here are the bounding boxes for the {plural}:\n"
        for ins_idx, ann_id in enumerate(image_category[id][cat]):
            ann_id = str(ann_id)
            bbox = annotations[ann_id]['bbox']
            x1, x2, y1, y2 = bbox["x1"], bbox["x2"], bbox["y1"], bbox["y2"]
            assert x1 >= 0 and x2>= 0 and y1>=0 and y2>=0
            assert x1 < 336 and x2 < 336 and y1 < 336 and y2 < 336
            answer += f"Bounding box {str(ins_idx+1)}: (<PIXEL_{str(x1)}>,<PIXEL_{str(y1)}>,<PIXEL_{str(x2)}>,<PIXEL_{str(y2)}>)\n"
        answer = answer[:-1]
        
    else:
        answer =f"There are no {plural} in the image."
    assert answer.count("<PIXEL_") == num*4
    
    qs = [f"Please perform bounding box prediction for each instance of {plural} in the image.", f"Find instances of {plural}, return the x1 y1 x2 y2 coordinates of each instance.", f"Find instances of {plural} in the image and return their bounding boxes.", f"Return the bounding boxes of {plural}.", f"Please locate all of the {plural} in this image."]
    q = random.choice(qs)
    if id in train_ids:
        question_dict[str(id)] = {"q": q, "a": answer, "filename": el['coco_url'].split("/")[-1]}
    

np.save("bbox_data.npy", question_dict)

In [21]:
### create CoT data



image_dict = {}
for el in data['images']:
    cur_id = el['id']
    image_dict[cur_id] = el

n = set()
bads = 0
import random
random.seed(0)
items = list(image_dict.items())
random.shuffle(items)
image_dict = dict(items)
from collections import OrderedDict
image_dict = OrderedDict(image_dict)
question_dict = {}
count_dict = {}
alots = 0
H, W = 336, 336
for id , el in image_dict.items():
    if id not in cot_ids:
        continue
    if id not in image_category:
        neg_cats = el['neg_category_ids']
        sample_cat = random.sample(neg_cats, 1)[0]
        object = obj_dict[sample_cat]
        plural =  plurals[object]
        # q = f"How many {plural} are in the image?"
        a = "0"
        num =0 
        if num in count_dict and count_dict[num] >= 1000:
            continue
        if num not in count_dict:
            count_dict[num] =1
        else:
            count_dict[num] +=1
    
    else:
        cats = list(image_category[id].keys())
        nums = [len(image_category[id][cat]) for cat in list(image_category[id].keys())]
        index =  np.argmax(nums)
        # import pdb;pdb.set_trace()
        num = nums[index]
        cat =  cats[index]
        ann_ids = image_category[id][cat]
        is_valid = all([str(t) in annotations for t in ann_ids])
        # import pdb;pdb.set_trace()
        # print(is_valid)
        if not is_valid:
            bads+=1
            continue
        object = obj_dict[cat]
        plural =  plurals[object]
        assert num == len(image_category[id][cat])
        # q = f"How many {plural} are in the image?"
        a = f"{str(num)}"
        if num > 15:
            alots +=1
            continue
        if num in count_dict and count_dict[num] >= 1000:
            continue
        if num not in count_dict:
            count_dict[num] =1
        else:
            count_dict[num] +=1

    if num !=0:
        answer = f"Based on the analysis, here are the bounding boxes for the {plural}:\n"
        for ins_idx, ann_id in enumerate(image_category[id][cat]):
            ann_id = str(ann_id)
            bbox = annotations[ann_id]['bbox']
            x1, x2, y1, y2 = bbox["x1"], bbox["x2"], bbox["y1"], bbox["y2"]
            assert x1 >= 0 and x2>= 0 and y1>=0 and y2>=0
            assert x1 < 336 and x2 < 336 and y1 < 336 and y2 < 336
          
            answer += f"Bounding box {str(ins_idx+1)}: (<PIXEL_{str(x1)}>,<PIXEL_{str(y1)}>,<PIXEL_{str(x2)}>,<PIXEL_{str(y2)}>)\n"
        answer += f"Since there are {num} distinct bounding boxes and masks, we conclude there are {num} {plural}."
        
        
    else:
        answer =f"There are no {plural} in the image. We conclude there are {num} {plural}."
    assert answer.count("<PIXEL_") == num*4

    
    question_dict[str(id)] = {"q": f"How many {plural} are in the image?", "a": str(num), "filename": el['coco_url'].split("/")[-1]}
    question_dict[str(id)+"cot"] = {"q": f"How many {plural} are in the image?.\n To solve this, let’s carefully analyze the image. First, identify the bounding boxes associated with the {plural}. After that, count how many distinct bounding boxes correspond to {plural}.", "a": answer, "filename": el['coco_url'].split("/")[-1]}
    

np.save("cot_data.npy", question_dict)


In [23]:
### create llava-style vqa data with bbox and cot data
import uuid
bbox_dict = np.load("bbox_data.npy", allow_pickle=True).item()
cot_dict= np.load("cot_data.npy", allow_pickle=True).item()

COCO_PATH = "coco/train2017"

bbox_data = []
for k, value in bbox_dict.items():
    question =  value['q']
    answer =  value['a']
    path = value['filename']
    path = f"{COCO_PATH}/{path}"
    new_entry = {
            "id": str(uuid.uuid4()),
            "image": path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"
                },
            ]
            }
    bbox_data.append(new_entry)

cot_data = []
for k, value in cot_dict.items():
    question =  value['q']
    answer =  value['a']
    path = value['filename']
    path = f"{COCO_PATH}/{path}"
    new_entry = {
            "id": str(uuid.uuid4()),
            "image": path,
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{question}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"
                },
            ]
            }
    cot_data.append(new_entry)

In [26]:
## make annealing data
import random
epochs = 10
total_length = 5000
random.seed(0)
percents = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2 , 0.1]
total_cot_size = 0
annealing_data =[]
for epoch in range(epochs):
    percent = percents[epoch]
    epoch_bbox_data = random.sample(bbox_data, int(total_length * percent) )
    cot_length = total_length - int(total_length * percent)
    repeat = cot_length//len(cot_data)
    epoch_cot_data = cot_data * repeat
    total_epoch_data = epoch_bbox_data + epoch_cot_data
    # print(len(epoch_bbox_data), len(epoch_cot_data))
    annealing_data += total_epoch_data
    total_cot_size+= len(epoch_cot_data)
    
    
    
print(len(annealing_data), total_cot_size)
    
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
write_json(annealing_data, "../Data/train_lvis_annealing_data.json")    

50000 22500
